In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.stats as spstats


In [2]:
pd.set_option('display.max_columns', 85)
pd.set_option('display.max_rows', 85)

In [3]:
df = pd.read_csv("C://DataScience//Regis24/unique_orders.csv")
df.drop(columns = ['ORDER_ID', 'CUSTOMER_ID', 'delay_duration','NR_ITEMS_UNIQUE', 'NR_ITEMS',
                   'total_orders', 'paytime', 'rank' ], inplace=True)


In [4]:
# Monday is 0 and Sunday is 6
df['PAYMENT_DATE'] = pd.to_datetime(df['PAYMENT_DATE'])
df['ORDER_DATE'] = pd.to_datetime(df['ORDER_DATE'])
df['PAYMENT_DATE_year'] = df['PAYMENT_DATE'].dt.year#2
df['PAYMENT_DATE_month'] = df['PAYMENT_DATE'].dt.month#3
df['PAYMENT_DATE_day'] = df['PAYMENT_DATE'].dt.day#2
df['PAYMENT_DATE_day_of_week'] = df['PAYMENT_DATE'].dt.dayofweek
df['PAYMENT_DATE_is_month_start'] = df['PAYMENT_DATE'].dt.is_month_start
df['PAYMENT_DATE_is_month_end'] = df['PAYMENT_DATE'].dt.is_month_end
df['ORDER_DATE_year'] = df['ORDER_DATE'].dt.year#2
df['ORDER_DATE_month'] = df['ORDER_DATE'].dt.month#3
df['ORDER_DATE_day'] = df['ORDER_DATE'].dt.day#2
df['ORDER_DATE_day_of_week'] = df['ORDER_DATE'].dt.dayofweek
df['ORDER_DATE_is_month_start'] = df['ORDER_DATE'].dt.is_month_start
df['ORDER_DATE_is_month_end'] = df['ORDER_DATE'].dt.is_month_end
df['PAYMENT_DATE_is_weekend'] = np.where(df['PAYMENT_DATE_day_of_week'].isin([5,6]), 1, 0)
df['ORDER_DATE_is_weekend'] = np.where(df['ORDER_DATE_day_of_week'].isin([5,6]), 1, 0)
df['PAYMENT_DATE_is_month_start'] = df['PAYMENT_DATE_is_month_start'].astype(str)
df['PAYMENT_DATE_is_month_end'] = df['PAYMENT_DATE_is_month_end'].astype(str)
df['ORDER_DATE_is_month_end'] = df['ORDER_DATE_is_month_end'].astype(str)
df['ORDER_DATE_is_month_start'] = df['ORDER_DATE_is_month_start'].astype(str)


In [5]:
df.drop(columns = ['PAYMENT_DATE','ORDER_DATE'], inplace=True)

In [6]:
# drop these rows having cart value above 1000
# from dataFrame
df = df.drop(df[df['CART_VALUE'] > 800].index)
df.shape


(1681198, 22)

### payment_duration 

In [7]:
df = df.drop(df[df['payment_duration'] > 200].index)
df.shape

(1672446, 22)

In [8]:
df['CART_VALUE']= round(df['CART_VALUE'])
len(df['CART_VALUE'].unique())

801

In [9]:
df.shape

(1672446, 22)

### SPLIT DATA INTO TRAIN AND TEST

In [10]:
#Splitting the dataset to Test and Train
from sklearn.model_selection import train_test_split

X = df.drop(['delay_category'], axis= 1)
y = df['delay_category']
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=.2)
X_train = X_train.copy()            

In [11]:
### Normalize the pay_duration, age and Cart_value

In [12]:
y_train = pd.DataFrame(y_train)

In [13]:
from sklearn import preprocessing
min_max = preprocessing.MinMaxScaler()

In [14]:
# copy the data
X_train_min_max_scaled = X_train.copy()
  
# apply normalization techniques by Column 
column = 'AGE'
X_train_min_max_scaled[column] = (X_train_min_max_scaled[column] - X_train_min_max_scaled[column].min()) / (X_train_min_max_scaled[column].max() - X_train_min_max_scaled[column].min())    
column = 'payment_duration'
X_train_min_max_scaled[column] = (X_train_min_max_scaled[column] - X_train_min_max_scaled[column].min()) / (X_train_min_max_scaled[column].max() - X_train_min_max_scaled[column].min())    
column = 'CART_VALUE'
X_train_min_max_scaled[column] = (X_train_min_max_scaled[column] - X_train_min_max_scaled[column].min()) / (X_train_min_max_scaled[column].max() - X_train_min_max_scaled[column].min())    


In [15]:
# Test data same steps:
X_train = X_train_min_max_scaled
X_train.shape

(1337956, 21)

In [16]:
from sklearn.preprocessing import LabelEncoder
# Instantiate LabelEncoder
le = LabelEncoder() 

In [17]:
cols = ['PAYMENT_DATE_month', 'PAYMENT_DATE_year', 'PAYMENT_DATE_day','PAYMENT_DATE_day_of_week',
        'ORDER_DATE_year','ORDER_DATE_month','ORDER_DATE_day','ORDER_DATE_day_of_week' ]

# Encode labels of multiple columns at once
X_train[cols] = X_train[cols].apply(le.fit_transform)


In [18]:
dummies_X_Train = pd.get_dummies(X_train[['PAYMENT_DATE_is_month_start', 'PAYMENT_DATE_is_month_end','ORDER_DATE_is_month_start',
        'ORDER_DATE_is_month_end','PAYMENT_DATE_is_weekend', 'ORDER_DATE_is_weekend','SHOP', 
        'PAYMENT_INFO', 'GENDER','Default_Flag']])

In [19]:
dummies_X_Train.shape

(1337956, 32)

In [20]:
dummies_X_Train.join(X_train[['AGE','payment_duration','CART_VALUE','PAYMENT_DATE_month', 'PAYMENT_DATE_year', 'PAYMENT_DATE_day','PAYMENT_DATE_day_of_week',
        'ORDER_DATE_year','ORDER_DATE_month','ORDER_DATE_day','ORDER_DATE_day_of_week' ]])

,PAYMENT_DATE_is_weekend,ORDER_DATE_is_weekend,Default_Flag,PAYMENT_DATE_is_month_start_False,PAYMENT_DATE_is_month_start_True,PAYMENT_DATE_is_month_end_False,PAYMENT_DATE_is_month_end_True,ORDER_DATE_is_month_start_False,ORDER_DATE_is_month_start_True,ORDER_DATE_is_month_end_False,ORDER_DATE_is_month_end_True,SHOP_Baby & Child,SHOP_Car Accessories,SHOP_DIY Superstore & Tools,SHOP_Electronics,SHOP_Fashion,SHOP_Food,SHOP_Furniture,SHOP_Jewellery,SHOP_Media,SHOP_Shoes,SHOP_Sports,SHOP_Toys,SHOP_Young Fashion,PAYMENT_INFO_CONTRACT_TERMINATION,PAYMENT_INFO_INKASSO_CLOSED,PAYMENT_INFO_INKASSO_PENDING,PAYMENT_INFO_PAID_FULL,PAYMENT_INFO_PAID_OVER,GENDER_f,GENDER_m,GENDER_u,AGE,payment_duration,CART_VALUE,PAYMENT_DATE_month,PAYMENT_DATE_year,PAYMENT_DATE_day,PAYMENT_DATE_day_of_week,ORDER_DATE_year,ORDER_DATE_month,ORDER_DATE_day,ORDER_DATE_day_of_week
575201,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.367521,0.085427,0.18500,1,3,20,3,3,1,2,6
797086,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0.495726,0.110553,0.05625,8,3,10,2,3,7,18,0
272238,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0.145299,0.040201,0.07500,3,2,3,2,2,2,25,0
912619,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0.299145,0.070352,0.34875,11,3,9,1,3,10,24,0
1519251,0,1,0,0,1,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.581197,0.010050,0.11125,11,4,0,1,4,10,27,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736625,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0.589744,0.055276,0.05750,6,3,8,1,3,5,26,3
200590,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0.376068,0.050251,0.11375,11,1,11,1,1,11,0,4
935804,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0.418803,0.100503,0.17250,11,3,30,1,3,11,9,1
1355074,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.410256,0.040201,0.06625,8,4,2,3,4,7,24,1


In [21]:
# X_train['AGE','payment_duration','CART_VALUE','PAYMENT_DATE_month', 'PAYMENT_DATE_year', 'PAYMENT_DATE_day','PAYMENT_DATE_day_of_week',
#         'ORDER_DATE_year','ORDER_DATE_month','ORDER_DATE_day','ORDER_DATE_day_of_week' ]

In [22]:
X_train_final = dummies_X_Train.join(X_train[['AGE','payment_duration','CART_VALUE','PAYMENT_DATE_month', 'PAYMENT_DATE_year', 'PAYMENT_DATE_day','PAYMENT_DATE_day_of_week',
        'ORDER_DATE_year','ORDER_DATE_month','ORDER_DATE_day','ORDER_DATE_day_of_week' ]])

In [23]:
X_train_final.shape

(1337956, 43)

In [24]:
X_train_columnlist = X_train.columns
X_train.columns

Index(['PAYMENT_INFO', 'payment_duration', 'SHOP', 'AGE', 'CART_VALUE',
       'GENDER', 'Default_Flag', 'PAYMENT_DATE_year', 'PAYMENT_DATE_month',
       'PAYMENT_DATE_day', 'PAYMENT_DATE_day_of_week',
       'PAYMENT_DATE_is_month_start', 'PAYMENT_DATE_is_month_end',
       'ORDER_DATE_year', 'ORDER_DATE_month', 'ORDER_DATE_day',
       'ORDER_DATE_day_of_week', 'ORDER_DATE_is_month_start',
       'ORDER_DATE_is_month_end', 'PAYMENT_DATE_is_weekend',
       'ORDER_DATE_is_weekend'],
      dtype='object')

In [25]:
# Encode labels of multiple columns at once

y_train = y_train.apply(le.fit_transform)

# Print head
y_train.head(3)

,delay_category
575201,0
797086,0
272238,4


# Test Data

In [26]:
# copy the data
# copy the data
X_test_min_max_scaled = X_test.copy()
  
# apply normalization techniques by Column 
column = 'AGE'
X_test_min_max_scaled[column] = (X_test_min_max_scaled[column] - X_test_min_max_scaled[column].min()) / (X_test_min_max_scaled[column].max() - X_test_min_max_scaled[column].min())    
column = 'payment_duration'
X_test_min_max_scaled[column] = (X_test_min_max_scaled[column] - X_test_min_max_scaled[column].min()) / (X_test_min_max_scaled[column].max() - X_test_min_max_scaled[column].min())    
column = 'CART_VALUE'
X_test_min_max_scaled[column] = (X_test_min_max_scaled[column] - X_test_min_max_scaled[column].min()) / (X_test_min_max_scaled[column].max() - X_test_min_max_scaled[column].min())    


In [27]:
# Test data same steps:
X_test = X_test_min_max_scaled
X_test.shape


(334490, 21)

In [28]:
cols = ['PAYMENT_DATE_month', 'PAYMENT_DATE_year', 'PAYMENT_DATE_day','PAYMENT_DATE_day_of_week',
        'ORDER_DATE_year','ORDER_DATE_month','ORDER_DATE_day','ORDER_DATE_day_of_week' ]

# Encode labels of multiple columns at once
X_test[cols] = X_test[cols].apply(le.fit_transform)

In [29]:
dummies_X_Test = pd.get_dummies(X_test[['PAYMENT_DATE_is_month_start', 'PAYMENT_DATE_is_month_end','ORDER_DATE_is_month_start',
        'ORDER_DATE_is_month_end','PAYMENT_DATE_is_weekend', 'ORDER_DATE_is_weekend','SHOP', 
        'PAYMENT_INFO', 'GENDER','Default_Flag']])

In [30]:
dummies_X_Train.shape

(1337956, 32)

In [31]:
dummies_X_Test.join(X_test[['AGE','payment_duration','CART_VALUE','PAYMENT_DATE_month', 'PAYMENT_DATE_year', 'PAYMENT_DATE_day','PAYMENT_DATE_day_of_week',
        'ORDER_DATE_year','ORDER_DATE_month','ORDER_DATE_day','ORDER_DATE_day_of_week' ]])

,PAYMENT_DATE_is_weekend,ORDER_DATE_is_weekend,Default_Flag,PAYMENT_DATE_is_month_start_False,PAYMENT_DATE_is_month_start_True,PAYMENT_DATE_is_month_end_False,PAYMENT_DATE_is_month_end_True,ORDER_DATE_is_month_start_False,ORDER_DATE_is_month_start_True,ORDER_DATE_is_month_end_False,ORDER_DATE_is_month_end_True,SHOP_Baby & Child,SHOP_Car Accessories,SHOP_DIY Superstore & Tools,SHOP_Electronics,SHOP_Fashion,SHOP_Food,SHOP_Furniture,SHOP_Jewellery,SHOP_Media,SHOP_Shoes,SHOP_Sports,SHOP_Toys,SHOP_Young Fashion,PAYMENT_INFO_CONTRACT_TERMINATION,PAYMENT_INFO_INKASSO_CLOSED,PAYMENT_INFO_INKASSO_PENDING,PAYMENT_INFO_PAID_FULL,PAYMENT_INFO_PAID_OVER,GENDER_f,GENDER_m,GENDER_u,AGE,payment_duration,CART_VALUE,PAYMENT_DATE_month,PAYMENT_DATE_year,PAYMENT_DATE_day,PAYMENT_DATE_day_of_week,ORDER_DATE_year,ORDER_DATE_month,ORDER_DATE_day,ORDER_DATE_day_of_week
251273,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.410714,0.135678,0.14875,2,2,19,1,2,1,19,1
44507,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0.491071,0.100503,0.48625,2,1,1,3,1,1,8,3
1118422,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0.267857,0.120603,0.24750,4,4,11,1,4,3,16,4
417873,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0.401786,0.085427,0.12625,8,2,17,1,2,7,30,4
1555624,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.312500,0.100503,0.43750,0,5,4,1,4,11,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277503,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0.267857,0.356784,0.05375,5,2,12,2,2,3,1,0
681272,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0.330357,0.040201,0.07000,4,3,20,1,3,4,11,6
1116060,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0.267857,0.065327,0.08750,3,4,27,1,4,3,13,1
49690,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0.098214,0.221106,0.11250,3,1,7,5,1,1,21,2


In [32]:
X_test_final = dummies_X_Test.join(X_test[['AGE','payment_duration','CART_VALUE','PAYMENT_DATE_month', 'PAYMENT_DATE_year', 'PAYMENT_DATE_day','PAYMENT_DATE_day_of_week',
        'ORDER_DATE_year','ORDER_DATE_month','ORDER_DATE_day','ORDER_DATE_day_of_week' ]])


In [33]:
X_test_final.shape

(334490, 43)

In [34]:
X_train_final.shape

(1337956, 43)

In [35]:
X_train_final.head(2)

,PAYMENT_DATE_is_weekend,ORDER_DATE_is_weekend,Default_Flag,PAYMENT_DATE_is_month_start_False,PAYMENT_DATE_is_month_start_True,PAYMENT_DATE_is_month_end_False,PAYMENT_DATE_is_month_end_True,ORDER_DATE_is_month_start_False,ORDER_DATE_is_month_start_True,ORDER_DATE_is_month_end_False,ORDER_DATE_is_month_end_True,SHOP_Baby & Child,SHOP_Car Accessories,SHOP_DIY Superstore & Tools,SHOP_Electronics,SHOP_Fashion,SHOP_Food,SHOP_Furniture,SHOP_Jewellery,SHOP_Media,SHOP_Shoes,SHOP_Sports,SHOP_Toys,SHOP_Young Fashion,PAYMENT_INFO_CONTRACT_TERMINATION,PAYMENT_INFO_INKASSO_CLOSED,PAYMENT_INFO_INKASSO_PENDING,PAYMENT_INFO_PAID_FULL,PAYMENT_INFO_PAID_OVER,GENDER_f,GENDER_m,GENDER_u,AGE,payment_duration,CART_VALUE,PAYMENT_DATE_month,PAYMENT_DATE_year,PAYMENT_DATE_day,PAYMENT_DATE_day_of_week,ORDER_DATE_year,ORDER_DATE_month,ORDER_DATE_day,ORDER_DATE_day_of_week
575201,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.367521,0.085427,0.18500,1,3,20,3,3,1,2,6
797086,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0.495726,0.110553,0.05625,8,3,10,2,3,7,18,0


In [36]:
X_test_final.head(2)

,PAYMENT_DATE_is_weekend,ORDER_DATE_is_weekend,Default_Flag,PAYMENT_DATE_is_month_start_False,PAYMENT_DATE_is_month_start_True,PAYMENT_DATE_is_month_end_False,PAYMENT_DATE_is_month_end_True,ORDER_DATE_is_month_start_False,ORDER_DATE_is_month_start_True,ORDER_DATE_is_month_end_False,ORDER_DATE_is_month_end_True,SHOP_Baby & Child,SHOP_Car Accessories,SHOP_DIY Superstore & Tools,SHOP_Electronics,SHOP_Fashion,SHOP_Food,SHOP_Furniture,SHOP_Jewellery,SHOP_Media,SHOP_Shoes,SHOP_Sports,SHOP_Toys,SHOP_Young Fashion,PAYMENT_INFO_CONTRACT_TERMINATION,PAYMENT_INFO_INKASSO_CLOSED,PAYMENT_INFO_INKASSO_PENDING,PAYMENT_INFO_PAID_FULL,PAYMENT_INFO_PAID_OVER,GENDER_f,GENDER_m,GENDER_u,AGE,payment_duration,CART_VALUE,PAYMENT_DATE_month,PAYMENT_DATE_year,PAYMENT_DATE_day,PAYMENT_DATE_day_of_week,ORDER_DATE_year,ORDER_DATE_month,ORDER_DATE_day,ORDER_DATE_day_of_week
251273,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.410714,0.135678,0.14875,2,2,19,1,2,1,19,1
44507,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0.491071,0.100503,0.48625,2,1,1,3,1,1,8,3


In [37]:
y_test = pd.DataFrame(y_test)

In [38]:
# Encode labels of multiple columns at once

y_test = y_test.apply(le.fit_transform)

# Print head
y_test.head(3)

,delay_category
251273,0
44507,0
1118422,0


### Check the final X and y and train and test

In [39]:
print (X_test_final.shape)
print (X_train_final.shape)
print (y_test.shape)
print (y_train.shape)

(334490, 43)
(1337956, 43)
(334490, 1)
(1337956, 1)


## MODEL CREATION 

In [50]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)



In [51]:
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train_final,y_train.values.ravel() )  



RandomForestClassifier()

In [52]:
y_pred=clf.predict(X_test_final)

In [53]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9999073215940686
